In [18]:
import pandas as pd
import os

from glob import glob

from sshtunnel import SSHTunnelForwarder

import paramiko
import geopandas as gpd
import numpy as np
import json
import rasterio
import sshtunnel
import MySQLdb

from sqlalchemy import (
    Connection,
    CursorResult,
    Engine,
    Select,
    TextClause,
    create_engine,
    func,
    select,
)

In [2]:
pkl_files = glob(os.path.join('./OSN_pickles', '**', '*.pkl'), recursive=True)

In [6]:
osn_data = pd.read_pickle(pkl_files[10])

In [7]:
osn_data.geoaltitude.unique()

array([266.7 , 259.08, 251.46, 243.84, 243.84, 236.22, 228.6 , 220.98,
       213.36, 205.74, 198.12, 190.5 , 190.5 , 182.88, 175.26, 167.64,
       160.02, 289.56, 281.94, 274.32, 266.7 , 259.08, 251.46, 274.32,
       297.18, 304.8 , 312.42, 320.04, 594.36, 601.98, 487.68, 495.3 ,
       480.06, 472.44, 441.96, 434.34, 426.72, 419.1 , 411.48, 403.86,
       396.24, 388.62, 381.  , 464.82, 457.2 , 472.44, 480.06, 449.58,
       358.14, 365.76, 373.38, 350.52, 342.9 , 335.28, 327.66, 464.82,
       373.38, 358.14, 365.76, 571.5 , 579.12, 563.88, 609.6 , 297.18,
       304.8 , 236.22, 228.6 , 220.98, 182.88, 175.26, 167.64, 152.4 ,
       312.42, 320.04, 327.66, 342.9 , 381.  , 388.62, 396.24, 746.76,
       678.18, 685.8 , 693.42, 701.04, 708.66, 716.28, 723.9 , 746.76,
       739.14, 731.52, 251.46, 434.34, 350.52, 335.28, 487.68, 495.3 ,
       502.92, 144.78, 137.16, 129.54, 121.92, 510.54, 510.54, 518.16,
       670.56, 662.94, 655.32, 647.7 , 640.08, 632.46, 624.84, 617.22,
      

In [9]:
gdf_files = glob(os.path.join('./dataframes', '*', 'gdf*.json'))  # First level
gdf_files.extend(glob(os.path.join('./dataframes', 'gdf*.json')))  # Base directory


In [10]:
gdf_files

['./dataframes/2022-08-01_2022-08-31/gdf_2022-08-01_2022-08-31.json',
 './dataframes/2024-01-01_2024-01-31/gdf_2024-01-01_2024-01-31.json',
 './dataframes/2023-08-08_2023-08-15/gdf_2023-08-08_2023-08-15.json',
 './dataframes/2021-10-01_2021-10-31/gdf_2021-10-01_2021-10-31.json',
 './dataframes/2024-07-01_2024-07-31/gdf_2024-07-01_2024-07-31.json',
 './dataframes/2021-07-01_2021-07-31/gdf_2021-07-01_2021-07-31.json',
 './dataframes/2023-07-16_2023-07-31/gdf_2023-07-16_2023-07-31.json',
 './dataframes/2022-12-01_2022-12-31/gdf_2022-12-01_2022-12-31.json',
 './dataframes/2021-05-01_2021-05-31/gdf_2021-05-01_2021-05-31.json',
 './dataframes/2021-11-01_2021-11-30/gdf_2021-11-01_2021-11-30.json',
 './dataframes/2023-04-01_2023-04-30/gdf_2023-04-01_2023-04-30.json',
 './dataframes/2024-02-01_2024-02-29/gdf_2024-02-01_2024-02-29.json',
 './dataframes/2023-09-01_2023-09-07_TO_CHECK/gdf_2023-09-01_2023-09-07.json',
 './dataframes/2021-02-01_2021-02-28/gdf_2021-02-01_2021-02-28.json',
 './datafra

In [11]:
gdf_data = pd.read_json(gdf_files[10])

In [12]:
gdf_data.head()

,time,icao24,lat,lon,velocity,heading,callsign,onground,squawk,baroaltitude,...,hour,ref,inf_flt,inf_pt,dist,gnd_elev,gnd_inf_flt,gnd_inf_pt,min_hgt,lidar_min_alt
0,1681498398,3cc004,50.919629,6.991653,92.669915,87.772953,UNI131,false,5571,746.76,...,1681495200,3cc004_1_140423,False,False,0.000000,49.901081,False,False,463.594711,460.57
1,1681498399,3cc004,50.919676,6.993179,92.669915,87.772953,UNI131,false,5571,739.14,...,1681495200,3cc004_1_140423,False,False,107.086772,49.591351,False,False,463.594711,460.57
2,1681498400,3cc004,50.919676,6.994247,92.155862,87.760524,UNI131,false,5571,731.52,...,1681495200,3cc004_1_140423,False,False,181.959847,49.971550,False,False,463.594711,460.57
3,1681498401,3cc004,50.919708,6.995544,91.641813,87.747956,UNI131,false,5571,731.52,...,1681495200,3cc004_1_140423,False,False,272.949299,50.352779,False,False,463.594711,460.57
4,1681498402,3cc004,50.919769,6.996841,91.149547,87.412121,UNI131,false,5571,723.90,...,1681495200,3cc004_1_140423,False,False,364.115178,51.000000,False,False,462.545879,461.48


In [13]:
gdf_data.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'callsign',
       'onground', 'squawk', 'baroaltitude', 'geoaltitude', 'lastposupdate',
       'lastcontact', 'hour', 'ref', 'inf_flt', 'inf_pt', 'dist', 'gnd_elev',
       'gnd_inf_flt', 'gnd_inf_pt', 'min_hgt', 'lidar_min_alt'],
      dtype='object')

In [15]:
gdf_data['positionnac'] = 10
gdf_data['version'] = 2



In [17]:

MYSQL_secrets_json = './mysql_secrets.json'


with open(MYSQL_secrets_json) as MYSQL_secrets:
  MYSQL_creds = json.load(MYSQL_secrets)


In [19]:
gdf_data.time.values[0]

1681498398

In [23]:
gdf_data = gdf_data.drop(columns=['hour', 'lidar_min_alt'])

KeyError: "['hour'] not found in axis"

In [25]:
ed25519_key = paramiko.Ed25519Key(filename='/home/dimitri/.ssh/id_ed25519')

# Set up the SSH tunnel with the RSA key
with SSHTunnelForwarder(
    (MYSQL_creds['SSH_ADDRESS'], 22),
    ssh_username=MYSQL_creds['SSH_USERNAME'],
    ssh_pkey=ed25519_key,  # Use the loaded RSA key
    remote_bind_address=(MYSQL_creds['REMOTE_BIND_ADDRESS'], MYSQL_creds['REMOTE_BIND_PORT']),
    allow_agent=False,
) as tunnel:
    
    print('connected')

    engstr =  'mysql+pymysql://' + MYSQL_creds['SSH_USERNAME'] + ':' + MYSQL_creds['PYANYWHERE_PASSWORD'] + '@127.0.0.1:' + str(tunnel.local_bind_port)+'/dme3$' + MYSQL_creds['TEST_DATABASE_NAME']

    engine = create_engine(engstr)

    print('step 1')
    
    gdf_data.to_sql(con=engine, name = MYSQL_creds['MAIN_TEST_TABLE_NAME'], if_exists='append')

    print("Done")

connected
step 1
Done
